# Time travel and snapshots

Iceberg's **time travel** feature enables you to query data as it existed in the past.
In this notebook, we'll explore:

* **Understanding snapshots**: What they are and how they work
* **Time travel queries**: Query data from specific points in time
* **Snapshot operations**: Inspect, compare, and manage snapshots
* **Rollback**: Undo changes by rolling back to previous snapshots
* **Branching and tagging**: Create named references to snapshots
* **Snapshot expiration**: Clean up old snapshots to save storage

By the end, you'll be able to use Iceberg's time travel features for debugging, auditing, and data recovery.

In [ ]:
import shutil
import daft
import pyarrow as pa
from pathlib import Path
from pyiceberg.catalog.sql import SqlCatalog
from datetime import datetime, timedelta
import time

%reload_ext autoreload
%autoreload 2
from helpers import compare_snapshots

## Setup: Create a table with rich history

Let's create a table with multiple snapshots to demonstrate time travel features.

In [ ]:
warehouse_path = Path('../data/warehouse_time_travel').absolute()
shutil.rmtree(warehouse_path, ignore_errors=True)
warehouse_path.mkdir(parents=True, exist_ok=True)
catalog_db = warehouse_path / 'catalog.db'
catalog_db.unlink(missing_ok=True)

catalog = SqlCatalog(
    'time_travel_demo',
    **{'uri': f'sqlite:///{catalog_db}', 'warehouse': f'file://{warehouse_path}'}
)
catalog.create_namespace('demo')
print("✅ Catalog initialized")

In [ ]:
df_events = daft.read_json('../data/input/events.jsonl')

# Snapshot 1: Initial load ("Week 1")
print("Creating Snapshot 1: Initial load")
df_week1 = df_events.limit(20000)
arrow_table = df_week1.to_arrow()
events_table = catalog.create_table('demo.events', schema=pa.schema(arrow_table.schema))
events_table.append(arrow_table)
time.sleep(1)  # Small delay to ensure snapshots have different timestamps

# Snapshot 2: "Week 2" data
print("Creating Snapshot 2: \"Week 2\" data")
df_week2 = df_events.offset(20000).limit(20000)
arrow_table = df_week2.to_arrow()
events_table.append(arrow_table)
time.sleep(1)  # Ensure distinct snapshot timestamps

# Snapshot 3: Delete "bad data" (discovered data quality issue)
print("Creating Snapshot 3: Delete OperationMode events (data quality fix)")
events_table.delete("type = 'OperationMode'")
time.sleep(1)  # Ensure distinct snapshot timestamps

# Snapshot 4: "Week 3" data
print("Creating Snapshot 4: \"Week 3\" data")
df_week3 = df_events.offset(40000).limit(20000)
arrow_table = df_week3.to_arrow()
events_table.append(arrow_table)

print(f"\n✅ Created table with {len(events_table.history())} snapshots")

## Understanding snapshots for time travel

Recall from the previous notebooks that each write operation creates a new **snapshot** - an immutable view of the table at that point in time. In the metadata deep dive, we explored how snapshots are stored in the metadata JSON with properties like `snapshot_id`, `timestamp_ms`, `manifest_list`, and operation `summary`.

For time travel, two additional snapshot properties become crucial:

* **parent_snapshot_id**: Links to the previous snapshot, forming a chain of history
* **sequence_number**: Monotonically increasing number that orders snapshots

This chain structure is what enables time travel:
* Walk backwards through history to any previous state
* Understand the exact order of operations
* Identify which snapshot was current at any given time

Let's inspect our snapshots and see this chain in action:

In [ ]:
# Get snapshot history
history = events_table.snapshots()

print(f"Total snapshots: {len(history)}\n")
print(f"{'Snap':<5} {'Timestamp':<20} {'Operation':<10} {'Records':<12} {'Files'}")
print("-" * 70)

for i, snapshot in enumerate(history, 1):
    timestamp = datetime.fromtimestamp(snapshot.timestamp_ms / 1000).strftime('%Y-%m-%d %H:%M:%S')
    operation = snapshot.summary.operation.value if snapshot.summary else 'N/A'

    # Get statistics from summary
    props = snapshot.summary.additional_properties if snapshot.summary else {}
    total_records = props.get('total-records', props.get('added-records', 'N/A'))
    total_files = props.get('total-data-files', props.get('added-data-files', 'N/A'))

    print(f"{i:<5} {timestamp:<20} {operation:<10} {str(total_records):<12} {total_files}")

In [ ]:
# Inspect a specific snapshot in detail
snap1 = history[3]
print(f"Snapshot 1 Details:\n")
print(f"ID: {snap1.snapshot_id}")
print(f"Timestamp: {datetime.fromtimestamp(snap1.timestamp_ms / 1000)}")
print(f"Sequence number: {snap1.sequence_number if hasattr(snap1, 'sequence_number') else 'N/A'}")
print(f"Parent snapshot: {snap1.parent_snapshot_id if hasattr(snap1, 'parent_snapshot_id') else 'None (first snapshot)'}")

if snap1.summary:
    print(f"\nSummary:")
    print(f"  Operation: {snap1.summary.operation.value}")
    if snap1.summary.additional_properties:
        for key, value in sorted(snap1.summary.additional_properties.items()):
            print(f"  {key}: {value}")

## Time travel queries

To query historical versions of your data, switch to a previous snapshot.

In [ ]:
# Query current state
df_current = daft.read_iceberg(events_table)
print("Current state:")
daft.sql("SELECT COUNT(*) as total_events FROM df_current").show()

In [ ]:
# Query Snapshot 1 ("Week 1")
snapshot1_id = history[0].snapshot_id
df_snap1 = daft.read_iceberg(events_table, snapshot_id=snapshot1_id)
print(f"\nSnapshot 1 (\"Week 1\") (ID: {snapshot1_id}):")
daft.sql("SELECT COUNT(*) as total_events FROM df_snap1").show()

In [ ]:
# Query Snapshot 2 ("Week 2")
snapshot2_id = history[1].snapshot_id
df_snap2 = daft.read_iceberg(events_table, snapshot_id=snapshot2_id)
print(f"\nSnapshot 2 (\"Week 2\") (ID: {snapshot2_id}):")
daft.sql("SELECT COUNT(*) as total_events FROM df_snap2").show()

Since each snapshot includes a timestamp, you can find the snapshot that was valid at a particular time as well. This is useful for questions like:

* "Show me the data as of yesterday at 3pm"
* "What did the monthly report see on March 1st?"

### Time travel use cases

Time travel is useful for:

1. **Debugging**: "What data did the broken job see?"
2. **Auditing**: "Show me all changes in the last 24 hours"
3. **Reproducing reports**: "Re-run the quarterly report with the exact data it used"
4. **Data recovery**: "Restore deleted records from yesterday"
5. **Testing**: "Compare results before and after a schema change"

Let's demonstrate a debugging scenario: We deleted OperationMode events in Snapshot 3. Let's verify that they existed before and are gone now.

In [ ]:
# Before deletion (Snapshot 2)
snap2_id = history[1].snapshot_id
df_before = daft.read_iceberg(events_table, snapshot_id=snap2_id)
print("Before deletion (Snapshot 2):")
daft.sql("""
    SELECT
        SUM(CASE WHEN type = 'OperationMode' THEN 1 ELSE 0 END) as operation_mode_events,
        COUNT(*) as total
    FROM df_before
""").show()

In [ ]:
# After deletion (Snapshot 3)
snap3_id = history[2].snapshot_id
df_after = daft.read_iceberg(events_table, snapshot_id=snap3_id)
print("\nAfter deletion (Snapshot 3):")
daft.sql("""
    SELECT
        SUM(CASE WHEN type = 'OperationMode' THEN 1 ELSE 0 END) as operation_mode_events,
        COUNT(*) as total
    FROM df_after
""").show()

print("\n✅ Time travel confirmed the deletion happened in Snapshot 3")

## Rollback: Undoing changes

Rollback reverts the table to a previous snapshot. This is like `git reset` - it makes a previous snapshot current.

**Important**: Rollback doesn't delete snapshots or data. It just changes which snapshot is "current".

**Key limitation**: Rollback only works with **ancestor snapshots** - you can only roll back to snapshots that are in the direct lineage of the current snapshot. Think of it like Git: you can't "rollback" to a future commit. This means rollback is designed for **recovering from mistakes** (undoing bad commits), not for freely navigating between arbitrary snapshot states. For navigation, use time travel queries instead.

### When to use rollback

* Bad data was loaded
* A bug in the ingestion pipeline
* Accidental deletion
* Testing: rollback after test, then re-apply operations to return to latest

Let's rollback to before the deletion:

In [ ]:
# Current state (after deletion)
df = daft.read_iceberg(events_table)
daft.sql("""
    SELECT
        SUM(CASE WHEN type = 'OperationMode' THEN 1 ELSE 0 END) as operation_mode_events,
        COUNT(*) as total
    FROM df
""").show()

In [ ]:
# Rollback to Snapshot 2 (before deletion)
snap2_id = history[1].snapshot_id
print(f"\nRolling back to Snapshot 2 (ID: {snap2_id})...")
print(f"Note: This changes the 'current' snapshot but doesn't delete snapshots 3 and 4.\n")
events_table.manage_snapshots().rollback_to_snapshot(snap2_id).commit()

In [ ]:
events_table = catalog.load_table('demo.events') # Reload the table to get the updated state
df = daft.read_iceberg(events_table)
result = daft.sql("""
    SELECT
        SUM(CASE WHEN type = 'OperationMode' THEN 1 ELSE 0 END) as operation_mode_events,
        COUNT(*) as total
    FROM df
""").show()

print("\n✅ Rollback successful! OperationMode events are back.")

In [ ]:
# Check history after rollback - use snapshots() for full snapshot objects
history = events_table.snapshots()
current_snap_id = events_table.current_snapshot().snapshot_id

print("Snapshot history after rollback:\n")
for i, snap in enumerate(history, 1):
    is_current = snap.snapshot_id == current_snap_id
    marker = " ← CURRENT" if is_current else ""
    timestamp = datetime.fromtimestamp(snap.timestamp_ms / 1000).strftime('%Y-%m-%d %H:%M:%S')
    print(f"Snapshot {i}: {snap.snapshot_id} @ {timestamp}{marker}")


Note: All snapshots still exist. We just changed which one is 'current'.

Since we rolled back for demonstration purposes, let's restore the table to its latest state.

In [ ]:
latest_snapshot_id = history[-1].snapshot_id
events_table.manage_snapshots().set_current_snapshot(
    snapshot_id=latest_snapshot_id
).commit()

## Branching and tagging

Iceberg supports **named references** to snapshots:

* **Tags**: Immutable pointers to snapshots (like Git tags)
  - Use for: releases, quarterly reports, milestones
  - Example: `quarterly-report-2024-Q4`

* **Branches**: Mutable pointers that can advance (like Git branches)
  - Use for: experimental changes, staging environments
  - Example: `experiment-new-schema`

Note: Contrary to Git, the scope of a tag or branch in the Iceberg standard is only a single table (unless you are using Project Nessie as catalog).

### Creating tags

Tags are useful for marking important snapshots that you want to keep forever.

In [ ]:
# Reload history and create tags for important snapshots
history = events_table.snapshots()
snap1_id = history[0].snapshot_id
snap2_id = history[1].snapshot_id

# Tag the initial load (snapshot_id, tag_name)
events_table.manage_snapshots().create_tag(snap1_id, 'initial-load').commit()
print(f"✅ Created tag 'initial-load' → Snapshot {snap1_id}")

# Tag week 2
events_table.manage_snapshots().create_tag(snap2_id, 'week-2-complete').commit()
print(f"✅ Created tag 'week-2-complete' → Snapshot {snap2_id}")

# Reload the table to get the updated state
events_table = catalog.load_table('demo.events')

# List available tags
print("\nAvailable tags in the table:")
for ref_name, ref in events_table.metadata.refs.items():
    print(f"  • {ref_name} → Snapshot {ref.snapshot_id}")
print()


In [ ]:
# Query by tag name
# Note: daft.read_iceberg() requires integer snapshot IDs, not tag names
# So we need to resolve the tag to its snapshot ID first
initial_load_ref = events_table.metadata.refs.get('initial-load')
if initial_load_ref:
    initial_load_snapshot_id = initial_load_ref.snapshot_id
    df_initial = daft.read_iceberg(events_table, snapshot_id=initial_load_snapshot_id)
    print(f"Querying 'initial-load' tag (snapshot {initial_load_snapshot_id}):")
    daft.sql("SELECT COUNT(*) as total FROM df_initial").show()
else:
    print("⚠️  Tag 'initial-load' not found")

In [ ]:
# Query another tag
week2_ref = events_table.metadata.refs.get('week-2-complete')
if week2_ref:
    week2_snapshot_id = week2_ref.snapshot_id
    df_week2 = daft.read_iceberg(events_table, snapshot_id=week2_snapshot_id)
    print(f"Querying 'week-2-complete' tag (snapshot {week2_snapshot_id}):")
    daft.sql("SELECT COUNT(*) as total FROM df_week2").show()
else:
    print("⚠️  Tag 'week-2-complete' not found")

### Creating branches

Branches allow you to make experimental changes without affecting the main timeline. Unlike tags, branches are mutable - new commits can be made to a branch.

This allows you to:

* Develop experimental features in isolation
* Test schema changes without affecting the main table
* Create staging environments for validation

In [ ]:
# Create a branch from the current snapshot
current_snapshot_id = events_table.current_snapshot().snapshot_id

events_table.manage_snapshots().create_branch(
    snapshot_id=current_snapshot_id,
    branch_name='experimental',
    min_snapshots_to_keep=2  # Keep at least 2 snapshots on this branch
).commit()

print(f"✅ Created branch 'experimental' from snapshot {current_snapshot_id}")

# Reload table and list all references (tags and branches)
events_table = catalog.load_table('demo.events')
print("\nAll references (tags and branches):")
for ref_name, ref in events_table.metadata.refs.items():
    ref_type = "branch" if ref_name not in ['main', 'initial-load', 'week-2-complete'] else "tag" if ref_name != 'main' else "main branch"
    print(f"  • {ref_name} ({ref_type}) → Snapshot {ref.snapshot_id}")

In [ ]:
# Query the branch - like tags, we need to resolve it to a snapshot ID
print("\nQuerying the 'experimental' branch:")
experimental_ref = events_table.metadata.refs.get('experimental')
if experimental_ref:
    experimental_snapshot_id = experimental_ref.snapshot_id
    df_experimental = daft.read_iceberg(events_table, snapshot_id=experimental_snapshot_id)
    print(f"Branch snapshot ID: {experimental_snapshot_id}")
    daft.sql("SELECT COUNT(*) as total FROM df_experimental").show()

In [ ]:
print("\n📝 Writing to the experimental branch...")
df_test = df_events.offset(60000).limit(1000)
test_data = df_test.to_arrow()

# Append to the experimental branch (not main)
events_table.append(test_data, branch='experimental')

print(f"✅ Appended {len(test_data)} records to 'experimental' branch")

# Reload and check the branches
events_table = catalog.load_table('demo.events')
print("\nAfter writing to experimental branch:")
main_ref = events_table.metadata.refs.get('main')
experimental_ref = events_table.metadata.refs.get('experimental')
print(f"  • main branch → Snapshot {main_ref.snapshot_id}")
print(f"  • experimental branch → Snapshot {experimental_ref.snapshot_id}")
if main_ref.snapshot_id != experimental_ref.snapshot_id:
    print("  ✓ Branches now point to different snapshots!")

# Query both branches to see the difference
df_main = daft.read_iceberg(events_table, snapshot_id=main_ref.snapshot_id)
df_exp = daft.read_iceberg(events_table, snapshot_id=experimental_ref.snapshot_id)

print("\nRecord counts:")
print(f"  Main branch: {daft.sql('SELECT COUNT(*) as count FROM df_main').to_pydict()['count'][0]:,}")
print(f"  Experimental branch: {daft.sql('SELECT COUNT(*) as count FROM df_exp').to_pydict()['count'][0]:,}")

## Snapshot expiration

Snapshots are retained forever by default. This enables unlimited time travel, but:

* **Storage costs**: Old data files accumulate
* **Metadata overhead**: Manifest lists grow
* **Cleanup complexity**: Hard to know what's still needed

**Snapshot expiration** in Iceberg has two phases:

1. **Expire snapshots**: Removes old snapshots from metadata
2. **Delete orphan files**: Removes unreferenced data files (separate operation)

PyIceberg currently implements **phase 1 only** - it removes snapshot metadata but leaves data files intact. Deleting orphan files is [currently under implementation](https://github.com/apache/iceberg-python/pull/1958).

Let's demonstrate snapshot metadata expiration only for now.

In [ ]:
print("Current snapshots in the table:")
history = events_table.snapshots()
for i, snap in enumerate(history, 1):
    timestamp = datetime.fromtimestamp(snap.timestamp_ms / 1000).strftime('%Y-%m-%d %H:%M:%S')
    is_tagged = any(ref.snapshot_id == snap.snapshot_id for ref in events_table.metadata.refs.values())
    tag_marker = " [TAGGED - Protected]" if is_tagged else ""
    print(f"  Snapshot {i}: {snap.snapshot_id} @ {timestamp}{tag_marker}")

print(f"\nTotal: {len(history)} snapshots")

# Run snapshot expiration
# This will remove snapshot metadata for old snapshots
# Tagged snapshots are automatically protected
print("\n🧹 Running snapshot metadata expiration...")

# Expire snapshots older than 2 seconds for demonstration purposes.
# Note: Protected snapshots (tags, branches) are automatically excluded
events_table.maintenance.expire_snapshots().older_than(
    datetime.now() - timedelta(seconds=2)
).commit()

# Reload and check what remains
events_table = catalog.load_table('demo.events')
history = events_table.snapshots()

print("Snapshots after expiration:")
for i, snap in enumerate(history, 1):
    timestamp = datetime.fromtimestamp(snap.timestamp_ms / 1000).strftime('%Y-%m-%d %H:%M:%S')
    is_tagged = any(ref.snapshot_id == snap.snapshot_id for ref in events_table.metadata.refs.values())
    tag_marker = " [TAGGED - Protected]" if is_tagged else ""
    print(f"  Snapshot {i}: {snap.snapshot_id} @ {timestamp}{tag_marker}")

print(f"\nTotal: {len(history)} snapshots")

## Review questions

**What role do `parent_snapshot_id` and `sequence_number` play in time travel?**
   - How does the parent chain enable walking backwards through history?
   - Why does Iceberg need both properties?

**When would you use time travel queries vs rollback?**
   - What's the difference in their purpose and effect?
   - Can you rollback to any snapshot?

**How is Iceberg time travel different from traditional backup files?**
   - Think about query capabilities, storage efficiency, and metadata overhead.
   - What advantages does the snapshot model provide?

**What happens to data files when you rollback?**
   - Does it affect the snapshot or data files?
   - Are files deleted or moved?

**What does deleting a tag actually remove?**
   - What actually changes in the metadata?
   - If you delete a tag, can you still access that snapshot?

**What is removed with snapshot expiration?**
   - What is actually removed?
   - What is safe?

**What's the difference between tags and branches in Iceberg?**
   - Compare it with Git.
   - What happens to the main branch when you write to a different branch?

**What could be reasons to separate metadata expiration from data deletion?**
   - What happens exactly in the phases?
   - What are the safety implications?
   - Can data files be orphaned in another way? Think of transactions.

## Challenges

### Simulate a bad load and recover

1. Query the current table and note the total record count
2. Append some test data to the events table
3. Verify the new data appears in the table
4. "Discover" it's bad data and rollback to the previous snapshot
5. Verify the table is back to the original state
6. **Bonus**: Can you "redo" the bad load by advancing to the future snapshot?

### Find which snapshot changed something

We deleted OperationMode events somewhere in our history:

1. Loop through all snapshots and query each one
2. Check how many OperationMode events exist at each snapshot
3. Identify exactly which snapshot performed the deletion
4. How many OperationMode events were deleted?
5. **Hint**: Compare the count before and after each snapshot

### Work with branches for isolated development

1. Create a new branch called `quality-check` from the current snapshot
2. Write some test data to the `quality-check` branch (don't modify main)
3. Query both `main` and `quality-check` branches
4. Compare the record counts to verify isolation
5. **Bonus**: What happens if you try to rollback the main branch to a snapshot on the quality-check branch?

### Compare snapshots

1. Use the `compare_snapshots()` helper function (already imported)
2. Compare the two tagged snapshots: 'initial-load' and 'week-2-complete'
3. What operations occurred between them?
4. How many files and records were added?

### Recovery from accidental deletion

Simulate accidentally deleting important records:

1. Delete all events where `type = 'Alarm'` from the main branch
2. Use time travel to query the previous snapshot and count how many Alarm events existed
3. Export those alarm events to a variable (hint: `.to_pydict()` or `.to_arrow()`)
4. Rollback to restore the table
5. **Bonus**: Instead of rollback, could you re-insert just the deleted records?

### Create an audit trail report

1. For each snapshot in the history, extract:
   - Timestamp
   - Operation type
   - Records added/deleted
   - Total records in that snapshot
2. Display as a formatted table or dataframe
3. Calculate: What's the net change in records from first to last snapshot?
4. **Bonus**: Create a visualization showing record count over time

### Understand expiration protection

After the expiration we ran earlier:

1. List which snapshots survived expiration and which were removed
2. Explain why each protected snapshot was not expired
3. What would happen if you:
   - Removed all tags?
   - Ran expiration with `older_than(datetime.now())`?
4. **Bonus**: Try creating a tag, running expiration, then removing the tag. Can you still access that snapshot?

## Summary

In this notebook, we explored Iceberg's time travel capabilities:

* **Snapshots**: Immutable views of the table at specific points in time
  - Created by every write operation
  - Form a chain with parent pointers
  - Contain operation summaries and statistics

* **Time travel**: Query historical data
  - By snapshot ID (precise)
  - By timestamp (user-friendly)
  - Use cases: debugging, auditing, reproducing reports

* **Rollback**: Undo changes
  - Non-destructive (snapshots remain)
  - Changes which snapshot is "current"
  - Can be undone by rolling back again

* **Tags and branches**: Named references
  - Tags: Immutable, for milestones
  - Branches: Mutable, for experiments
  - PyIceberg supports writing to branches directly (since 0.10)
  - Enable human-readable snapshot references

* **Snapshot expiration**: Clean up old metadata
  - PyIceberg: Removes snapshot metadata only, but currently cannot clean data files (use Spark for that or wait for the pull request :-) ).
  - Time-based or count-based policies
  - Respects tags and retention settings

### Key takeaways

1. **Time travel is cheap**: Just metadata changes, no data copying
2. **Rollback is safe**: Can always roll forward again
3. **Tags preserve history**: Tagged snapshots never expire
4. **Branches enable isolation**: Write experimental changes without affecting main
5. **Metadata vs data cleanup**: PyIceberg separates these for safety
6. **Audit trail is automatic**: Every change is recorded

Next, we will look at schema evolution.
